## Start and prep

In [1]:
# we will use Kaggle dataset from here:
# https://www.kaggle.com/datasets/blastchar/telco-customer-churn
# code and video from here:
# https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/03-classification/01-churn-project.md
# notebook 
# https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/03-classification/notebook.ipynb
# dataset for download
# https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv

# Data preparation

# Download the data, read it with pandas
# Look at the data
# Make column names and values look uniform
# Check if all the columns read correctly
# Check if the churn variable needs any preparation
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'


In [3]:
# run once to download the dataset
# !wget $data -O data-week-3.csv 

In [4]:
df = pd.read_csv('data-week-3.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Data cleaning part

In [5]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [6]:
df.head().T # nice trick to see all colums ))

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [7]:
# df.dtypes # totalcharges  object - it should be a number! Fill missing values with zeroes...
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce') # convedrting to numbers and replace "-" with NaN

In [8]:
df.totalcharges = df.totalcharges.fillna(0) # fill NaN with zeroes

In [9]:
# df.totalcharges.isnull().sum() # np.int64(0) - now it is OK - no NaN here, only digits
# df.dtypes # totalcharges float64 - all good!
# but churn is the onject still - churn  object - it has yes/no text marks
# df.churn
df.churn = (df.churn == 'yes').astype(int) 

In [10]:
# df.churn # now all looks good -we converted yes to 1 and no to 0

## Setting up the validation framework using Scikit-Learn

In [11]:
# to split dataset automatically we will use scikit-learn
from sklearn.model_selection import train_test_split

In [12]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [13]:
# lets see how skl split our dataset
len(df_train), len(df_val), len(df_test) # looks good - (4225, 1409, 1409)

(4225, 1409, 1409)

In [14]:
# we need to reset index in each split for simplicity
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [15]:
# and remove label we predict - so our model will not accidentally look at it
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

del df_train['churn']
del df_val['churn']
del df_test['churn']

In [25]:
# offtopic - what if I forgot what train_test_split function is doing? Built-in help:
# train_test_split?

## EDA

In [16]:
# We have to
# -- Check missing values
# -- Look at the target variable (churn)
# -- Look at numerical and categorical variables

df_full_train = df_full_train.reset_index(drop=True)

In [23]:
# df_full_train.isnull().sum() # all good - zeroes everywhere - no missing values ))

# we have not deleted our label from full_train - so we can have a look at stats:
df_full_train.churn.value_counts(normalize=True)

# churn - using only df_full_train.churn.value_counts()
# 0    4113 - customers stayed - 73%
# 1    1521 - churned customers - 26%
# churn - using df_full_train.churn.value_counts(normalize=True)
# 0    0.730032
# 1    0.269968

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

In [54]:
global_churn = df_full_train.churn.mean() # np.float64(0.26996805111821087)
global_churn
# this is called CHURN RATE = 0.2699... or 27% in our case...
# mean works here because churn variable is binary - consists of 1 and 0
# churned customers are marked by 1 in our case

np.float64(0.26996805111821087)

In [41]:
# we have only 3 numeric variables, rest is categorical
numerical = ['tenure', 'monthlycharges', 'totalcharges']
# df_full_train.dtypes
# categorical = [c for c in list(df_full_train.columns) if c not in numerical]
# categorical

# on video it was done manually ))))
# df_full_train.columns
categorical = [ 'gender', 'seniorcitizen', 'partner', 'dependents',
        'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']
# categorical

In [42]:
# how many values each categorical variable has:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

## Feature importance for categorical features

In [45]:
# Idea is to look at churn rate in different groups - to see which categorical variable can predict it

# Feature importance: Churn rate and risk ratio
# Feature importance analysis (part of EDA) - identifying which features affect our target variable

# -- Churn rate
# -- Risk ratio
# -- Mutual information - later

# for example gender does not predict churn:
df_full_train[df_full_train.gender == 'female'].churn.mean()
# np.float64(0.27682403433476394) - roughly the same as in the parent group

np.float64(0.27682403433476394)

In [48]:
# we can repeat the same analysis for other categorical variables
df_full_train.partner.value_counts()

partner
no     2932
yes    2702
Name: count, dtype: int64

In [52]:
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean() # 20% - np.float64(0.20503330866025166)
churn_partner

np.float64(0.20503330866025166)

In [53]:
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean() # 33% - np.float64(0.3298090040927694)
churn_no_partner
# churn rate is bigger but not sure if it is statistically significant?

np.float64(0.3298090040927694)

In [55]:
# risk ratios - our partner/no partner categorical variables explains ~20% of churn:
churn_partner / global_churn # less by 24% in group who lives with partner
# np.float64(0.7594724924338315)

np.float64(0.7594724924338315)

In [56]:
churn_no_partner / global_churn # churn risk is higher by ~22% in a no partner people group
# np.float64(1.2216593879412643)

np.float64(1.2216593879412643)

In [57]:
# risk analysis code in SQL:
# SELECT
#     gender,
#     AVG(churn),
#     AVG(churn) - global_churn AS diff,
#     AVG(churn) / global_churn AS risk
# FROM
#     data
# GROUP BY
#     gender;


# same code in Pandas to analyse risk ratios of every categorical feature
from IPython.display import display

for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)
    print()
    print()


gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980




seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208




partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730




paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560




paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


## Feature importance - mutual information